# Optimizing data access and processing

Over the multiple formats presented in the other tutorials, some were presented as cloud optimized. This notebook shows some methods to take advantage of those formats to optimize data access and processing. The methods presented are specifically optimized for cloud usage but can (and should most of the time) also be used with local files. The general idea is to limit the data access: either by loading only the required data or opening small fractions (chunks) of a whole dataset and iterate over all these chunks.

Some examples in this notebook use data downloaded or generated in other notebooks. Make sure you ran them before running this one. They also provide useful information and context on the files and their formats.


## Raster data

In the [raster formats tutorial](./raster_formats.ipynb), COGs (Cloud Optimized Geotiffs) were presented as the best cloud-oriented solution, as they are natively split and compressed in chunks and allow overviews. In this section, we present how to load and process a raster chunk by chunk. This is particularly usefull when using large datasets which can't be loaded in memory (both on local or cloud architectures) or when accessing a specific sub area of a temporal series of images.

### Using rasterio

Reading chunks with rasterio is done using a window: a smaller section of the image, defined by an offset and a size on the x and y axes. A simple way to implement reading an image chunk by chunk is using chunks with the same width as the image (i.e. one or multiple lines) and iterating over them.

In [1]:
import rasterio
from rasterio.windows import Window

cog_path = "./sample_data/rasters/data/xt_SENTINEL2B_20180621-111349-432_L2A_T30TWT_D_V1-8_RVBPIR_cog.tif"

In [2]:
# read the entire raster (for comparison)
with rasterio.open(cog_path) as src:
        %time data = src.read(1)

# read chunk by chunk
chunk_size = 500  # number of lines per chunk
with rasterio.open(cog_path) as src:
    width = src.width
    height = src.height
    for start_row in range(0, height, chunk_size):  # first row of the chunk
        # last row of the chunk = first line + chunk size
        # the last chunk may have less lines than chunk_size => height is used instead 
        end_row = min(start_row + chunk_size, height)
        window = Window(0, start_row, width, end_row - start_row)
        %time data = src.read(1, window=window)  # read the 1st band
        print(data.shape)

CPU times: user 85.8 ms, sys: 35 ms, total: 121 ms
Wall time: 125 ms
CPU times: user 20.5 ms, sys: 5.02 ms, total: 25.5 ms
Wall time: 25.5 ms
(500, 2410)
CPU times: user 18.8 ms, sys: 5.99 ms, total: 24.8 ms
Wall time: 24.8 ms
(500, 2410)
CPU times: user 18.8 ms, sys: 5.91 ms, total: 24.7 ms
Wall time: 24.7 ms
(500, 2410)
CPU times: user 21.5 ms, sys: 2.04 ms, total: 23.5 ms
Wall time: 23.5 ms
(500, 2410)
CPU times: user 3.39 ms, sys: 971 µs, total: 4.36 ms
Wall time: 4.37 ms
(80, 2410)


COGs have an intern chunk size used when writing them. It is easier and usually more efficient to use this chunk size instead, using `src.block_windows`. In this code `ji` is the current chunk's coordinates, relative to the chunk grid. The coordinates of the first pixel of the chunk are given by the widow's column and row offset.

In [3]:
with rasterio.open(cog_path) as src:
    for ji, window in src.block_windows(1):
        print(f"Chunk's coordinates: {ji}")
        print(window)
        data = src.read(window=window)
        print(data.shape, "\n")

Chunk's coordinates: (0, 0)
Window(col_off=0, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 1)
Window(col_off=256, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 2)
Window(col_off=512, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 3)
Window(col_off=768, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 4)
Window(col_off=1024, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 5)
Window(col_off=1280, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 6)
Window(col_off=1536, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 7)
Window(col_off=1792, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 8)
Window(col_off=2048, row_off=0, width=256, height=256)
(4, 256, 256) 

Chunk's coordinates: (0, 9)
Window(col_off=2304, row_off=0, width=106, height=256)
(4, 256, 106) 

Chunk's coordina

### Using rioxarray

Rasterio opens raster as numpy arrays and processes them sequentially. Instead, it is possible to use `xioxarray`, a library to open rasters as xarrays:

In [4]:
import rioxarray as rxr

xds = rxr.open_rasterio(cog_path, chunks=True)
xds

<xarray.DataArray (band: 4, y: 2080, x: 2410)> Size: 80MB
dask.array<open_rasterio-df2a314bbacb95f006457625f718f7b3<this-array>, shape=(4, 2080, 2410), dtype=float32, chunksize=(1, 2080, 2410), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 19kB 5.013e+05 5.013e+05 ... 5.253e+05 5.253e+05
  * y            (y) float64 17kB 5.279e+06 5.279e+06 ... 5.258e+06 5.258e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

The `rioxarray` [official documentation](https://corteva.github.io/rioxarray/stable/examples/read-locks.html#Chunking) details the chunking process and how to optimize chunk management. This also allows the use of `dask` to process chunks in parallel. Once again, the official documentation has details on how to use dask with rioxarray: https://corteva.github.io/rioxarray/stable/examples/dask_read_write.html.

## Vector data

Vector data allows two types of optimizations:

- streaming data chunk by chunk (similar to raster files)
- filtering input data before loading it in memory

These optimizations are not necesarily available for all vector formats. Geoparquet is the most suitable format for these uses: it can be easily streamed and filtered. 

### Streaming

First, let's try reading data chunk by chunk. This can be easily done using `pyarrow`. To illustrate this example, we'll re-use the `landuse.parquet` file generated in the [vector formats tutorial](./vector_data_formats.ipynb) and count the number of forest polygons in each chunk (or batch).

In [5]:
import pyarrow.parquet as pq
file_path = "./sample_data/vector/departement-31/landuse.parquet"

parquet_file = pq.ParquetFile(file_path)

for n, batch in enumerate(parquet_file.iter_batches(batch_size=1000)):  # batch_size: number of rows to read
    print(f"Batch number {n}, n_rows: {batch.num_rows}")
    column=batch.column("type").to_pandas()  # select the column 'type' and transform the column to a pandas series
    n_forest = len(column[column == "forest"].index)  # count the occurrences of 'forest' 
    print(f"{n_forest} occurences of 'forest'\n")

Batch number 0, n_rows: 1000
316 occurences of 'forest'

Batch number 1, n_rows: 1000
273 occurences of 'forest'

Batch number 2, n_rows: 1000
515 occurences of 'forest'

Batch number 3, n_rows: 1000
245 occurences of 'forest'

Batch number 4, n_rows: 1000
286 occurences of 'forest'

Batch number 5, n_rows: 1000
421 occurences of 'forest'

Batch number 6, n_rows: 1000
423 occurences of 'forest'

Batch number 7, n_rows: 1000
437 occurences of 'forest'

Batch number 8, n_rows: 1000
184 occurences of 'forest'

Batch number 9, n_rows: 1000
75 occurences of 'forest'

Batch number 10, n_rows: 1000
28 occurences of 'forest'

Batch number 11, n_rows: 1000
11 occurences of 'forest'

Batch number 12, n_rows: 1000
11 occurences of 'forest'

Batch number 13, n_rows: 1000
15 occurences of 'forest'

Batch number 14, n_rows: 1000
26 occurences of 'forest'

Batch number 15, n_rows: 297
49 occurences of 'forest'



## Datacube

Datacube formats (and the corresponding libraries) are generally better optimized for the cloud. Let's reuse the `.zarr` file generated in the [data cube formats tutorial](./datacube_formats.ipynb). As explained in that tutorial, `.zarr` files are natively split into chunks to facilitate chunking mechanisms. Another optimization is lazy loading and computing. Lazy loading means that the data is not loaded into memory until needed; only the metadata is stored. This allows the user to define operations that, similarly, are not computed until required. The loading and computing steps are then optimized and executed at the end, improving ressource usage. Here is an example:

In [6]:
import hvplot.xarray
import xarray as xr
zarr_path = "./sample_data/data_cube/example_from_xarray.zarr"

da = xr.open_dataset(zarr_path)["data"]
da.variable

/home/eplanteu/dev/pluto/pluto-tuto-11-format-donnees/data_types_env/lib/python3.11/site-packages/xarray/backends/api.py:652: RuntimeWarning: 'netcdf4' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)
/home/eplanteu/dev/pluto/pluto-tuto-11-format-donnees/data_types_env/lib/python3.11/site-packages/xarray/backends/api.py:652: RuntimeWarning: 'scipy' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)


<xarray.Variable (time: 10, lat: 1000, lon: 1000)> Size: 80MB
[10000000 values with dtype=float64]

The actual values of the array aren't loaded. One way of explicitly loading them into memory is using the `.load()` method:

In [7]:
da.load()
da.variable

<xarray.Variable (time: 10, lat: 1000, lon: 1000)> Size: 80MB
array([[[0.57173056, 0.44281942, 0.16513337, ..., 0.80223243,
         0.78291964, 0.08747034],
        [0.31521199, 0.41597532, 0.49439683, ..., 0.35265298,
         0.60145573, 0.3083475 ],
        [0.20306074, 0.20359435, 0.33797216, ..., 0.26089714,
         0.76296845, 0.6778943 ],
        ...,
        [0.25245384, 0.5276968 , 0.52644091, ..., 0.58426732,
         0.09180941, 0.34140939],
        [0.45087638, 0.46311388, 0.52009816, ..., 0.40293499,
         0.26637495, 0.22535514],
        [0.42322559, 0.72216987, 0.37084262, ..., 0.77784775,
         0.12880648, 0.56547993]],

       [[0.88587595, 0.43882129, 0.40775621, ..., 0.78691236,
         0.84506516, 0.05957236],
        [0.2105488 , 0.12635273, 0.69809241, ..., 0.92477286,
         0.77313677, 0.17230435],
        [0.94823774, 0.53079968, 0.79555541, ..., 0.47326325,
         0.77537247, 0.14510511],
...
        [0.9682806 , 0.29507893, 0.75662358, ..., 0.46322352,
         0.13547584, 0.9072647 ],
        [0.62804832, 0.59621295, 0.11512536, ..., 0.31490865,
         0.8390884 , 0.46678536],
        [0.22960135, 0.85382707, 0.62060189, ..., 0.30999343,
         0.1842668 , 0.28276561]],

       [[0.92283423, 0.76045385, 0.82228549, ..., 0.98505589,
         0.60402087, 0.89849785],
        [0.17968167, 0.88254461, 0.3432763 , ..., 0.62870849,
         0.66852378, 0.2862194 ],
        [0.40432042, 0.70899042, 0.34822044, ..., 0.8682068 ,
         0.13327541, 0.13511111],
        ...,
        [0.5277495 , 0.30094765, 0.51537951, ..., 0.38737032,
         0.86513829, 0.44451509],
        [0.35835956, 0.51757299, 0.22708403, ..., 0.50319819,
         0.06690598, 0.2319543 ],
        [0.91434051, 0.30306714, 0.28184485, ..., 0.12336382,
         0.31688197, 0.56079195]]])

Let's reopen it to reset it to lazy loaded and test lazy computing. For this, let's multiply the array by 10 and compute its mean over all time periods:

In [8]:
da = xr.open_dataset(zarr_path, chunks="auto", engine="zarr")["data"]

print("Lazy loaded array:")  # lazy loaded
print(da, "\n\n")

scaled_data = da * 10
mean_data = scaled_data.mean(dim="time")  # lazy computed
print("Lazy computed array:")
print(mean_data, "\n\n")


result = mean_data.compute()  # load in memory and compute
print("Loaded and computed:")
print(result)

Lazy loaded array:
<xarray.DataArray 'data' (time: 10, lat: 1000, lon: 1000)> Size: 80MB
dask.array<open_dataset-data, shape=(10, 1000, 1000), dtype=float64, chunksize=(10, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * lon      (lon) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * time     (time) int64 80B 0 1 2 3 4 5 6 7 8 9 


Lazy computed array:
<xarray.DataArray 'data' (lat: 1000, lon: 1000)> Size: 8MB
dask.array<mean_agg-aggregate, shape=(1000, 1000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * lon      (lon) int64 8kB 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999 


Loaded and computed:
<xarray.DataArray 'data' (lat: 1000, lon: 1000)> Size: 8MB
array([[5.75989822, 5.02703079, 3.19270113, ..., 6.09194011, 5.12143591,
        3.23814132],
       [3.7197

As explained in the rasters section, an efficient optimization method is to run chunks computation in parallel, using dask for example. And `xarray` uses dask to handle data arrays, making it easier to assemble these methods.

## Conclusion

There are many optimization techniques available for all the data types and formats presented, usable both on local and cloud architectures. It is important to consider which format and what processing methods can and should be used, depending on the architecture and ressources available, as well as the data volume. Generally, a modern and optimized code should be considered, but over a  small dataset, using parallel processing and chunking data may introduce unnecessary complexity and might be inefficient.